# Telco Churn II: this time it's causal

In [104]:
import pandas as pd
import numpy as np

import dowhy.api

In [125]:
df = pd.read_csv('../../data/churn.csv').drop(['customerID','TotalCharges'], axis='columns')

Specify a mapping from yes/no responses to integer values.

In [126]:
default_options = {'Yes': 1, 'No': 0}
internet_options = {'Yes': 1, 'No': 0, 'No internet service': -1}
phone_options = {'Yes': 1, 'No': 0, 'No phone service': -1}
gender_options = {'Female': 1, 'Male': 0}
contract_options = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
service_options = {'No': 0, 'DSL': 1, 'Fiber optic': 2}
payment_options = {'Mailed check': 0, 'Electronic check': 1, 'Credit card (automatic)': 2, 'Bank transfer (automatic)': 3}

In [65]:
column_options = {
    'Churn': default_options,
    'Partner': default_options,
    'Dependents': default_options,
    'PhoneService': default_options,
    'PaperlessBilling': default_options,
    'OnlineSecurity': internet_options,
    'OnlineBackup': internet_options,
    'DeviceProtection': internet_options,
    'TechSupport': internet_options,
    'StreamingTV': internet_options,
    'StreamingMovies': internet_options,
    'MultipleLines': phone_options,
    'gender': gender_options,
    'InternetService': service_options,
    'Contract': contract_options,
    'PaymentMethod': payment_options
}

In [75]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [142]:
# b = binary
# c = continuous
# d = discrete

variable_types = {
    'gender': 'b',
    'SeniorCitizen': 'b',
    'Partner': 'b',
    'Dependents': 'b',
    'tenure': 'c',
    'PhoneService': 'd',
    'MultipleLines': 'd',
    'InternetService': 'd',
    'OnlineSecurity': 'd',
    'OnlineBackup': 'd',
    'DeviceProtection': 'd',
    'TechSupport': 'd',
    'StreamingTV': 'd',
    'StreamingMovies': 'd',
    'Contract': 'd',
    'PaperlessBilling': 'd',
    'PaymentMethod': 'd',
    'MonthlyCharges': 'c',
    'TotalCharges': 'c',
    'Churn': 'b'
}

In [143]:
# convert binary variables

def prep(df):
    df = df.copy()
    
    for col, opt in column_options.items():
        df[col] = df[col].map(opt)

    return df

In [144]:
churn_df = prep(df)

In [145]:
model = 'digraph {  gender -> PhoneService;  gender -> InternetService;  gender -> OnlineSecurity;  gender -> OnlineBackup;  gender -> DeviceProtection;  gender -> TechSupport;  gender -> StreamingTV;  gender -> StreamingMovies;  gender -> Contract;  gender -> PaperlessBilling;  gender -> PaymentMethod;  gender -> MonthlyCharges;  gender -> Churn;  SeniorCitizen -> PhoneService;  SeniorCitizen -> InternetService;  SeniorCitizen -> OnlineSecurity;  SeniorCitizen -> OnlineBackup;  SeniorCitizen -> DeviceProtection;  SeniorCitizen -> TechSupport;  SeniorCitizen -> StreamingTV;  SeniorCitizen -> StreamingMovies;  SeniorCitizen -> Contract;  SeniorCitizen -> PaperlessBilling;  SeniorCitizen -> PaymentMethod;  SeniorCitizen -> MonthlyCharges;  SeniorCitizen -> Churn;  Partner -> PhoneService;  Partner -> InternetService;  Partner -> OnlineSecurity;  Partner -> OnlineBackup;  Partner -> DeviceProtection;  Partner -> TechSupport;  Partner -> StreamingTV;  Partner -> StreamingMovies;  Partner -> Contract;  Partner -> PaperlessBilling;  Partner -> PaymentMethod;  Partner -> MonthlyCharges;  Partner -> Churn;  Dependents -> PhoneService;  Dependents -> InternetService;  Dependents -> OnlineSecurity;  Dependents -> OnlineBackup;  Dependents -> DeviceProtection;  Dependents -> TechSupport;  Dependents -> StreamingTV;  Dependents -> StreamingMovies;  Dependents -> Contract;  Dependents -> PaperlessBilling;  Dependents -> PaymentMethod;  Dependents -> MonthlyCharges;  Dependents -> Churn;  PhoneService -> MultipleLines;  PhoneService -> DeviceProtection;  PhoneService -> Contract;  PhoneService -> MonthlyCharges;  InternetService -> OnlineSecurity;  InternetService -> OnlineBackup;  InternetService -> DeviceProtection;  InternetService -> TechSupport;  InternetService -> StreamingTV;  InternetService -> Contract;  InternetService -> PaperlessBilling;  InternetService -> MonthlyCharges;  StreamingTV -> StreamingMovies;  StreamingTV -> MonthlyCharges;  StreamingMovies -> MonthlyCharges;  OnlineSecurity -> MonthlyCharges;  OnlineBackup -> MonthlyCharges;  DeviceProtection -> MonthlyCharges;  Contract -> MonthlyCharges;  TechSupport -> MonthlyCharges;  TechSupport -> Churn;  PaymentMethod -> Churn;  tenure -> Churn;}'

In [147]:
churn_df.causal.do(
    x='TechSupport',
    outcome='Churn',
    variable_types=variable_types,
    dot_graph=model,
    proceed_when_unidentifiable=True)

ERROR:dowhy.causal_graph:Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['TechSupport'] on outcome ['Churn']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['Partner', 'Dependents', 'gender', 'SeniorCitizen', 'U']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.do_sampler:Using WeightingSampler for do sampling.
INFO:dowhy.do_sampler:Caution: do samplers assume iid data.


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn,propensity_score,weight
0,1,0,1,0,67,1,1,2,0,1,...,0,1,1,1,1,3,105.70,0,0.449503,2.224681
1,0,0,0,0,38,1,1,1,1,1,...,1,0,0,0,0,1,69.95,0,0.245664,4.070605
2,0,0,0,1,35,1,0,2,0,0,...,0,1,0,1,1,1,85.15,1,0.398307,2.510625
3,1,1,1,0,57,1,1,2,0,1,...,0,0,1,0,0,2,94.00,0,0.689392,1.450553
4,1,1,1,0,34,1,0,1,0,0,...,1,1,0,1,0,3,64.20,0,0.273973,3.649998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,1,1,33,1,0,0,-1,-1,...,-1,-1,-1,1,1,3,19.45,0,0.304753,3.281348
7039,0,0,1,1,50,1,0,0,-1,-1,...,-1,-1,-1,1,1,0,20.35,0,0.304753,3.281348
7040,1,0,1,1,23,1,1,1,1,0,...,1,1,0,2,1,3,77.15,0,0.362025,2.762243
7041,0,0,0,0,61,1,1,2,1,0,...,0,1,1,1,0,1,104.00,0,0.522241,1.914825


In [148]:
tech_support_on = churn_df.causal.do(
    x={'TechSupport': 1},
    outcome='Churn',
    variable_types=variable_types,
    dot_graph=model,
    proceed_when_unidentifiable=True
)

tech_support_on = churn_df.causal.do(
    x={'TechSupport': 0},
    outcome='Churn',
    variable_types=variable_types,
    dot_graph=model,
    proceed_when_unidentifiable=True
)

ERROR:dowhy.causal_graph:Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['TechSupport'] on outcome ['Churn']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['Partner', 'Dependents', 'gender', 'SeniorCitizen', 'U']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.do_sampler:Using WeightingSampler for do sampling.
INFO:dowhy.do_sampler:Caution: do samplers assume iid data.
ERROR:dowhy.causal_graph:Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...
INFO:dowhy.causal_graph:If thi

In [149]:
(tech_support_on['Churn'] - tech_support_off['Churn']).mean()

In [150]:
male = churn_df.causal.do(
    x={'gender': 0},
    outcome='Churn',
    variable_types=variable_types,
    dot_graph=model,
    method='mcmc',
    proceed_when_unidentifiable=True
)

ERROR:dowhy.causal_graph:Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['gender'] on outcome ['Churn']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['U']
INFO:dowhy.causal_identifier:Continuing by ignoring these unobserved confounders because proceed_when_unidentifiable flag is True.
INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.do_sampler:Using McmcSampler for do sampling.


Exception: Unrecognized variable type: d

In [151]:
from dowhy.causal_model import CausalModel

In [154]:
cm = CausalModel(
    data=churn_df,
    treatment='gender',
    outcome='Churn',
    graph=model
)

ERROR:dowhy.causal_graph:Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...
INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['gender'] on outcome ['Churn']


In [157]:
estimable = cm.identify_effect()

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['U']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n]  y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


In [159]:
print(estimable)

Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                        
─────────(Expectation(Churn))
d[gender]                    
Estimand assumption 1, Unconfoundedness: If U→{gender} and U→Churn then P(Churn|gender,,U) = P(Churn|gender,)
### Estimand : 2
Estimand name: iv
No such variable found!



In [165]:
print(cm.estimate_effect(estimable, method_name='backdoor.propensity_score_weighting'))

Exception: Propensity Score Weighting method is only applicable for binary treatments. Try explictly setting dtype=bool for the treatment column.